In [27]:
import json
import pandas as pd
import os
import re
from bisect import bisect_left

In [28]:
# Paths

pilot_docs = '/data/rsg/nlp/juanmoo1/projects/04_polymer/00_annotation/brat-v1.3_Crunchy_Frog/data/polymer/pilot_dylan/'

In [29]:
def get_annotated_document_names(ann_path):
    fnames = [e for e in os.listdir(ann_path) if e.lower().endswith(".ann") or e.lower().endswith(".txt")]
    names = sorted(set([e.split('.')[0] for e in fnames]))
    
    names = [e for e in names if (f'{e}.txt' in fnames) and (f'{e}.ann' in fnames)]
    names.sort()
    
    return names

In [30]:
def parse_document_annotations(doc_name, ann_dir_path=pilot_docs):
    
    text_doc_path = os.path.join(ann_dir_path, f'{doc_name}.txt')
    text = open(text_doc_path, 'r').read()
    
    ann_doc_path = os.path.join(ann_dir_path, f'{doc_name}.ann')
    ann_lines = open(ann_doc_path, 'r').readlines()
    
    tokens = text.split()
    tok_lens = [0] + list(map(len, tokens))
    for j in range(1, len(tok_lens)):
        tok_lens[j] += tok_lens[j - 1] + 1
        
    entities = []
    links = []

    
    for line in ann_lines:
        line = re.sub('\s+', ' ', line).split()

        ann_type = line[0]
        ann_name = line[1]
        
        if ann_type.startswith('T'):
            
            chr_start, chr_stop = map(int, line[2:4])
            ann_text = line[4]

            tok_start = bisect_left(tok_lens, chr_start)
            tok_stop = bisect_left(tok_lens, chr_stop)
            
            entities.append((ann_type, ann_name, tok_start, tok_stop))

        elif ann_type.startswith('R'):
            arg_1, arg_2 = map(lambda s: s.split(':')[1], line[2:4])
            links.append((ann_type, ann_name, arg_1, arg_2))

    
    return {
        "text": text,
        "spans": entities,
        "links": links
    }

In [31]:
document_names = get_annotated_document_names(pilot_docs)
parsed_annotations = [parse_document_annotations(dname) for dname in document_names]

In [17]:
with open("../data/extractions.json", 'w') as f:
    json.dump(parsed_annotations, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '../01_ner_bert/data/extractions.json'

In [25]:
os.listdir('../data')

[]